# Part 2: Introduction to Umami and the `Residual` Class

Umami is a package for calculating metrics for use with for Earth surface dynamics models. This notebook is the second notebook in a three-part introduction to using umami.

## Scope of this tutorial

Before starting this tutorial, you should have completed [Part 1: Introduction to Umami and the `Metric` Class](IntroductionToMetric.ipynb).

In this tutorial you will learn the basic principles behind using the `Residual` class to compare models and data using terrain statistics. 

If you have comments or questions about the notebooks, the best place to get help is through [GitHub Issues](https://github.com/TerrainBento/umami/issues).

To begin this example, we will import the required python packages. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from io import StringIO
import numpy as np
from landlab import RasterModelGrid, imshow_grid
from umami import Residual

## Step 1 Create grids

Unlike the first notebook, here we need to compare model and data. We will create two grids, the `model_grid` and the `data_grid` each with a field called field called `topographic__elevation` to it. Both are size (10x10). The `data_grid` slopes to the south-west, while the `model_grid` has some additional noise added to it. 

First, we construct and plot the `data_grid`.

In [ ]:
data_grid = RasterModelGrid((10, 10))
data_z = data_grid.add_zeros("node", "topographic__elevation")
data_z += data_grid.x_of_node + data_grid.y_of_node

imshow_grid(data_grid, data_z)

Next, we construct and plot `model_grid`. It differs only in that it has random noise added to the core nodes. 

In [ ]:
np.random.seed(42)

model_grid = RasterModelGrid((10, 10))
model_z = model_grid.add_zeros("node", "topographic__elevation")
model_z += model_grid.x_of_node + model_grid.y_of_node
model_z[model_grid.core_nodes] += np.random.randn(model_grid.core_nodes.size)
imshow_grid(model_grid, model_z)

We can difference the two grids to see how they differ. As expected, it looks like normally distributed noise. 

In [ ]:
imshow_grid(model_grid, data_z - model_z, cmap="seismic")

This example shows a difference map with 64 residuals on it. A more realistic application with a much larger domain would have tens of thousands. Methods of model analysis such as calibration and sensitivity analysis need model output, such as the topography shown here, to be distilled into a smaller number of values. This is the task that umami facilitates. 

## Step 2: Construct an umami `Residual`

Similar to constructing a `Metric`, a residual is specified by a dictionary or YAML-style input file. 

Here we repeat some of the content of the prior notebook:

Each calculation gets its own unique name (the key in the dictionary), and is associated with a value, a dictionary specifying exactly what should be calculated. The only value of the dictionary required by all umami calculations is `_func`, which indicates which of the [`umami.calculations`](https://umami.readthedocs.io/en/latest/umami.calculations.html) will be performed. Subsequent elements of this dictionary are the required inputs to the calculation function and are described in their documentation. 

Note that some calculations listed in the [`umami.calculations`](https://umami.readthedocs.io/en/latest/umami.calculations.html) submodule are valid for both the umami `Metric` and `Residual` classes, while others are for `Residual`s only (the `Metric` class was covered in [Part 1](IntroductionToMetric.ipynb) of this notebook series). 

The order that calculations are listed is read in as an [OrderedDict](https://docs.python.org/3/library/collections.html#collections.OrderedDict) and retained as the "calculation order". 

In our example we will use the following dictionary: 

```python
residuals = {
    "me": {
        "_func": "aggregate",
        "method": "mean",
        "field": "topographic__elevation"
    },
    "ep10": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 10
    }
}
```
This specifies calculation of the mean of `topographic__elevation` (to be called "me") and the 10th percentile `topographic__elevation` (called "ep10"). The equivalent portion of a YAML input file would look like:

```yaml
residuals:
    me:
        _func: aggregate
        method: mean
        field: topographic__elevation
    ep10:
        _func: aggregate
        method: percentile
        field: topographic__elevation
        q: 10
```

The following code constructs the `Residual`. Note that the only difference with the prior notebook is that instead of specifying only one grid, here we provide two. Under the hood umami checkes that the grids are compatible and will raise errors if they are not.

In [ ]:
residuals = {
    "me": {
        "_func": "aggregate",
        "method": "mean",
        "field": "topographic__elevation"
    },
    "ep10": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 10
    }
}
residual = Residual(model_grid, data_grid, residuals=residuals)

To calculate the residuals, run the `calculate` bound method. 

In [ ]:
residual.calculate()

Just like `Metric` classes, the `Residual` has some usefull methods and attributes. 

`residual.names` gives the names as a list, in calculation order. 

In [ ]:
residual.names

`residual.values` gives the values as a list, in calculation order. 

In [ ]:
residual.values

And a function is available to get the value of a given metric.

In [ ]:
residual.value("me")

## Step 5: Write output

The methods for writing output avaiable in `Metric` are also provided by `Residual`.

In [ ]:
out = StringIO()
residual.write_residuals_to_file(out, style="dakota")
file_contents = out.getvalue().splitlines()
for line in file_contents:
    print(line.strip())

In [ ]:
out = StringIO()
residual.write_residuals_to_file(out, style="yaml")
file_contents = out.getvalue().splitlines()
for line in file_contents:
    print(line.strip())

# Next steps

Now that you have a sense for how the `Metric` and `Residual` classes are used, try the next notebook: [Part 3: Other IO options (using umami without Landlab or terrainbento)](OtherIO_options.ipynb).